In [ ]:
import cutlass
import cutlass.cute as cute

from categories import *
from test_utils import *
from layout_utils import *


# Layout Categories

## Tuple morphisms

A tuple morphism $f:S \to T$ is encoded as an object of the class Tuple_morphism. For example:

In [149]:
f = Tuple_morphism(domain   = (256,256,512,512),
                   codomain = (256,2,512,4,512),
                   map      = (1,0,5,3))
print("f =",f)

g = random_Tuple_morphism()
print("g =",g)

f = (256, 256, 512, 512):(256, 2, 512, 4, 512):(1, 0, 5, 3)
g = (9, 5, 2, 1, 4):(2, 4):(0, 0, 1, 0, 2)


If $f$ is a tuple morphism, then we can compute the associated flat layout $L_f$:

In [150]:
@cute.jit
def example():
    f = Tuple_morphism(domain   = (256,256,512,512),
                       codomain = (256,2,512,4,512),
                       map      = (1,0,5,3))
    L_f = compute_flat_layout(f)
    print("L_f = ",L_f)
example()

L_f =  (256,256,512,512):(1,0,1048576,512)


If $L$ is a flat layout, we can check if $L$ is tractable, and if so, produce a tuple morphism $f$ with $L_f = L$.

In [151]:
@cute.jit
def example():
    L = cute.make_layout((2,2,2),stride = (8,2,512))
    f = compute_Tuple_morphism(L)
    print(f"{'f':<4}= {f}")
    print(f"{'L':<4}= {L}")
    print(f"{'L_f':<4}= {compute_flat_layout(f)}")
example()

f   = (2, 2, 2):(2, 2, 2, 2, 32, 2):(4, 2, 6)
L   = (2,2,2):(8,2,512)
L_f = (2,2,2):(8,2,512)


## Operations on tuple morphisms

### Composition

If $f:S \to T$ and $g:T \to U$ are tuple morphisms, then we can compose $f$ and $g$ to form the tuple morphism $g \circ f: S \to U$:

In [152]:
f = Tuple_morphism((2,2),(10,2,10,2),(2,4))
g = Tuple_morphism((10,2,10,2),(10,10,2,2),(1,3,2,4))
composite = f.compose(g)
composite.name = "g o f"
print("g o f =",composite)

g o f = (2, 2):(10, 10, 2, 2):(3, 4)


Composition of tuple morphisms is compatible with composition of layouts, in that

$L_{g \circ f} = L_g \circ L_f$:

In [154]:
@cute.jit
def example():
    f = Tuple_morphism((8,8,8,2),(8,2,8,4,8),(3,1,5,2))
    g = Tuple_morphism((8,2,8,4,8),(8,8,2,2),(1,3,2,0,0))
    composite = f.compose(g)
    layout_composite = compute_flat_layout(composite)
    composite_layout = cute.composition(compute_flat_layout(g),compute_flat_layout(f))
    print(f"{'L_{g o f}':<10}= {layout_composite}")
    print(f"{'L_g o L_f':<10}= {composite_layout}")
example()

L_{g o f} = (8,8,8,2):(8,1,0,64)
L_g o L_f = (8,8,8,2):(8,1,0,64)


### Sort

We can sort tuple morphisms:

In [155]:
f = Tuple_morphism((16,8,8,32,256),(2,2,8,16,8,4,32),(4,3,5,7,0),name = "f")
f_sorted = f.sort()
print(f"{'f':<7} = {f}")
print(f"{'sort(f)':<7} = {f_sorted}")

f       = (16, 8, 8, 32, 256):(2, 2, 8, 16, 8, 4, 32):(4, 3, 5, 7, 0)
sort(f) = (256, 8, 16, 8, 32):(2, 2, 8, 16, 8, 4, 32):(0, 3, 4, 5, 7)


### Coalesce

We can coalesce tuple morphisms:

In [156]:
f = Tuple_morphism((2,3,2,3,10),(4,2,3,3,2,3,10),(2,3,5,6,7))
f_coalesced = f.coalesce()
print(f"{'f':<11} = {f}")
print(f"{'coalesce(f)':<11} = {f_coalesced}")

f           = (2, 3, 2, 3, 10):(4, 2, 3, 3, 2, 3, 10):(2, 3, 5, 6, 7)
coalesce(f) = (6, 60):(4, 6, 3, 60):(2, 4)


This operation is compatible with the coalesce operation on layouts, in that $L_{\text{coalesce}(f)} = \text{coalesce}(L_f)$.

In [157]:
@cute.jit
def example():
    f             = Tuple_morphism((2,2,3,3,5,5),(4,2,2,3,3,10,5,5),(2,3,5,4,7,8),name = "f")
    f_coalesced   = f.coalesce()
    L_f           = compute_flat_layout(f)
    L_f_coalesced = compute_flat_layout(f_coalesced)
    print(f"{'L_coalesce(f)':<12} = {L_f_coalesced}")
    print(f"{'coalesce(L_f)':<12} = {cute.coalesce(L_f)}")
example()

L_coalesce(f) = (4,3,3,25):(4,48,16,1440)
coalesce(L_f) = (4,3,3,25):(4,48,16,1440)


### Concatenate

We can concatenate tuple morphisms $f$ and $g$ with the same codomain and disjoint images.

In [158]:
f = Tuple_morphism((10,10),(5,10,5,10,5,10),(6,4))
g = Tuple_morphism((5,5,5),(5,10,5,10,5,10),(3,1,0))
concatenated_morphism = f.concat(g)
print(f"{'f':<11}={f}")
print(f"{'g':<11}={g}")
print(f"{'concat(f,g)'}={concatenated_morphism}")

f          =(10, 10):(5, 10, 5, 10, 5, 10):(6, 4)
g          =(5, 5, 5):(5, 10, 5, 10, 5, 10):(3, 1, 0)
concat(f,g)=(10, 10, 5, 5, 5):(5, 10, 5, 10, 5, 10):(6, 4, 3, 1, 0)


### Complement

We can form the complement $f^*$ of a tuple morphism $f$:

In [159]:
f = Tuple_morphism((5,5),(2,5,4,5),(2,4))
f_complement = f.complement()
print(f"{'f':<13}={f}")
print(f"{'complement(f)'}={f_complement}")


f            =(5, 5):(2, 5, 4, 5):(2, 4)
complement(f)=(2, 4):(2, 5, 4, 5):(1, 3)


### Restrict

If $f:S \to T$ is a tuple morphism, then we can restrict $f$ to any subtuple of $T$:

In [160]:
f = Tuple_morphism((128,128,512,512,3),(4,512,128,512,128,3),(3,5,2,4,6))
print(f.restrict((1,2,3)))
print(f.restrict((3,5)))
print(f.restrict((4,)))
print(f.restrict(()))

(128, 128, 512):(4, 512, 128, 512, 128, 3):(3, 5, 2)
(512, 3):(4, 512, 128, 512, 128, 3):(2, 6)
(512,):(4, 512, 128, 512, 128, 3):(4,)
():(4, 512, 128, 512, 128, 3):()


### Factorize

If $f:S \to T$ is a tuple morphism, then we can factor $f$ through any subtuple of $T$ which contains the image of $f$:

In [161]:
f = Tuple_morphism((4,4,16),(4,16,4,8,16),(3,0,2))
print(f.factorize((1,2,3,4)))
print(f.factorize((2,3)))

(4, 4, 16):(4, 16, 4, 8):(3, 0, 2)
(4, 4, 16):(16, 4):(2, 0, 1)


## Nested tuples

A nested tuple $S$ is either an integer, or a tuple of nested tuples. For example:

In [162]:
S_1 = NestedTuple(64)
S_2 = NestedTuple((8,8,32))
S_3 = NestedTuple((((8,),),(2,(2,2))))
S_4 = NestedTuple((((),(5,)),((()),())))
S_5 = random_NestedTuple()
print("S_1 =",S_1)
print("S_2 =",S_2)
print("S_3 =",S_3)
print("S_4 =",S_4)
print("S_5 =",S_5)

S_1 = 64
S_2 = (8,8,32)
S_3 = (((8)),(2,(2,2)))
S_4 = (((),(5)),((),()))
S_5 = (((((4),(9),(6),(8))),8))


In [184]:
f = random_Nest_morphism()
print(f)


Nest_morphism :
------------------------------------
Domain     = ()
Codomain   = ((2,5,6),((3,8,7,1,1)))
Map        = ()



Every nested tuple S has a length, a rank and a size. For example:

In [64]:
S = NestedTuple(((16,),(((),),),((2,4,(8,2)),),(2,(2,))))
print(f"{'S':<10} = {S}")
print(f"{'length(S)':<10} = {S.length()}")
print(f"{'rank(S)':<10} = {S.rank()}")
print(f"{'size(S)':<10} = {S.size()}")



S          = ((16),((())),((2,4,(8,2))),(2,(2)))
length(S)  = 7
rank(S)    = 4
size(S)    = 8192


Nested tuples have entries and modes. For example:

In [65]:
S = NestedTuple(((16,),(((),),),((2,4,(8,2)),),(2,(2,))))

for i in range(1,S.length()+1):
    print(f"{'entry_'+str(i)+'(S)':<10} = {S.entry(i)}")

for i in range(1,S.rank()+1):
    print(f"{'mode_'+str(i)+'(S)':<10} = {S.mode(i)}")

entry_1(S) = 16
entry_2(S) = 2
entry_3(S) = 4
entry_4(S) = 8
entry_5(S) = 2
entry_6(S) = 2
entry_7(S) = 2
mode_1(S)  = (16)
mode_2(S)  = ((()))
mode_3(S)  = ((2,4,(8,2)))
mode_4(S)  = (2,(2))


We can substitute the values of a nested tuple with any other list of values. For example: 

In [66]:
S = NestedTuple(((16,),(((),),),((2,4,(8,2)),),(2,(2,))))
values = (10,10,2,2,5,5,5)
print(S.sub(values))

((10),((())),((10,2,(2,5))),(5,(5)))


As a special case of this construction, the profile of a nested tuple is

In [67]:
S = NestedTuple(((16,),(((),),),((2,4,(8,2)),),(2,(2,))))
print(S.profile())

((0),((())),((0,0,(0,0))),(0,(0)))


## Nested tuple morphisms

A nested tuple morphism $f:S \to T$ is encoded as an object of the class Nest_morphism. For example:

In [139]:
S = NestedTuple(((2,(2,2)),(((3,),),),((),1024)))
T = NestedTuple(((1024,(2,)),(2,(2,)),(3,((3,),)))) 
map = (4,2,0,6,1)
f = Nest_morphism(S,T,map)
print(f)

g = random_Nest_morphism()
print(g)


Nest_morphism :
-------------------------------------------
Domain     = ((2,(2,2)),(((3))),((),1024))
Codomain   = ((1024,(2)),(2,(2)),(3,((3))))
Map        = (4, 2, 0, 6, 1)


Nest_morphism :
--------------------------------------------
Domain     = (((9,((6,3)),((5,5)))),5,(3,5))
Codomain   = ((1,4,(5,5),9),(5,6))
Map        = (0, 7, 0, 3, 6, 0, 0, 0)



If $f$ is a nested tuple morphism, then we can compute the associated layout $L_f$:

In [142]:
@cute.jit
def example():
    f = Nest_morphism(domain   = NestedTuple( (((8,8),(16,16)), (32,32))),
                       codomain = NestedTuple( (((8,),(32,)),((8,),(16,)))),
                       map      = (1,3,0,4,0,2) )
    L_f = compute_layout(f)
    print("L_f = ",L_f)
example()

L_f =  (((8,8),(16,16)),(32,32)):(((1,256),(0,2048)),(0,8))


If $L$ is a layout, we can check if $L$ is tractable, and if so, produce a nested tuple morphism $f$ with $L_f = L$:

In [24]:
@cute.jit
def example():
    L = cute.make_layout((2,((),),(2,2)),stride = (8,((),),(2,512)))
    f = compute_Nest_morphism(L)
    print(f)
    print(f"{'L':<4}= {L}")
    print(f"{'L_f':<4}= {compute_layout(f)}")
example()


Nest_morphism :
---------------------------
Domain     = (2,(()),(2,2))
Codomain   = (2,2,2,2,32,2)
Map        = (4, 2, 6)

L   = (2,(()),(2,2)):(8,(()),(2,512))
L_f = (2,(()),(2,2)):(8,(()),(2,512))
